### SBAE notebook series - Part I - Sample size determination

The code snippets in this notebook will give a first idea of expected deforestation within a predefined Area of Interest based on Global Forest Cover data (Hansen et al 2013) for a given period of time.

Based on those deforestation estimates we can calculate the sample size based on an expected sampling error. In afirst step we apply the sample size calculation from Cochran.

Now that we have a rough idea of how much samples are needed, and thus how dense we need to sample our area, in order to arrive at a certain error, we re-run a sample simulation on the GFC data itself to see how much the expected error differs there.

Both analysis shall give us confidence in selecting an adequte sample size tat we can use in the following notebook, where we the actual sampling design is going to be created.

- todo
- write output to markdown to create a document
- 


In [ ]:
import ee
ee.Initialize()

import numpy as np
import pandas as pd
from retrying import retry

In [2]:
# period of change
start_year = 2015
end_year = 2020

# forest definition
tree_cover = 20         # in percent
mmu = 0.5               # in hectare

# aoi (various options)

# based on earth engine feature collection
country = 'Papua New Guinea'
gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")
aoi = gaul.filter(ee.Filter.eq("ADM0_NAME", country)).union()

# any ogr/gdal readable vectorfile (e.g. shp, gpkg, geojson, kml)
#local_file = 'path/to/local/file'

In [3]:
import numpy as np
def apply_MMU(image, mmu=5):
    
    mask = image.gt(0).connectedPixelCount(ee.Number(mmu).add(2)).gte(ee.Number(mmu));
    return image.updateMask(mask);

def mmu_in_pixels(ha, scale):
    
    sqm = ha/100*1000*1000
    sqm_scale = scale*scale
    mmu = np.floor(sqm/sqm_scale)
    print(mmu)
    return mmu
    
def get_aoi_areas(aoi, collection, start=2001, end=2022, tree_cover=20, mmu=0.5):
    
    if collection == 'CCI':

        lc = ee.ImageCollection("users/amitghosh/sdg_module/esa/cci_landcover")
        lc_start = lc.filter(ee.Filter.eq("system:index", f'{start}')).first()
        lc_end = lc.filter(ee.Filter.eq("system:index", f'{end}')).first()
        change = lc_start.neq(lc_end).clip(aoi)
        scale = 300

    elif collection == 'GFC':
        
        scale = 30
        hansen = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")
        loss = hansen.select('lossyear').unmask(0)
        change = loss.gte(ee.Number(start).subtract(2000)).And(loss.lte(ee.Number(end).subtract(2000)))
        forest = hansen.select('treecover2000').updateMask(
            loss.gte(ee.Number(start).subtract(2000)).Or(loss.eq(0))
        ).gte(tree_cover).unmask(0)
        
        
    # -----------------------------------------------------------------
    # getting total area of change
    total_image = forest.addBands(change).addBands(ee.Image(1)).multiply(ee.Image.pixelArea()).rename(['forest_area', 'change_area', 'total_area'])
    areas = total_image.reduceRegion(**{
            'reducer': ee.Reducer.sum(),
            'geometry': aoi,
            'scale': scale,
            'maxPixels': 1e14
        })
    
    
    d, areas = {}, areas.getInfo()
    for area in areas.keys():
        d[area] = np.round(areas[area]/1000000,2)
    
    span = end-start+1
    print(f"According to the GFC product, the Area of Interest covers an area of {d['total_area']} square kilometers,"
          f" of which {d['forest_area']} square kilometers have been forested in {start} ({np.round(d['forest_area']/d['total_area']*100, 2)} %)."
          f" Between {start} and {end}, {d['change_area']} square kilometers have been deforested."
          f" That corresponds to {d['change_area']/span} square kilometers of annual deforestation in average."    
    )
    
    return d

#area_dict = get_aoi_areas(aoi, 'GFC', start_year, end_year, tree_cover)
#yearly_dict = get_yearly_deforest_stats(aoi, 'GFC')


According to the GFC product, the Area of Interest covers an area of 1129313.5 square kilometers, of which 161807.49 square kilometers have been forested in 2015 (14.33 %). Between 2015 and 2020, 2233.28 square kilometers have been deforested. That corresponds to 372.21333333333337 square kilometers of annual deforestation in average.

In [ ]:
def calculate_sample_size_cochran(total_area, subarea, z_score=1.95, target_precision=0.1):
    
    # we calculate our proportion of change 
    proportional_change = subarea/total_area
    
    # our relative error
    error = target_precision * proportional_change
    
    # the upper term of Cochran (SE???)
    se = z_score*z_score * proportional_change * (1 - proportional_change) 
    
    return se/(error**2)


# we should invert that to do something similar as extrating from global data
def calculate_error_cochran(total_area, subarea, z_score=1.96, sample_size=250):
    
    proportional_change = subarea/total_area
    
    se = np.sqrt(proportional_change * (1 - proportional_change) / sample_size)
    
    ci = z_score * se
    
    return ci/proportional_change*100

def add_stats(row, actual_proportional):
    
    #abs_errors = [np.abs(np.subtract(i, actual_proportional)) for i in row['proportional_changes_sampled']]
    #mean_deviation, sd_deviation = np.nanmean(abs_errors), np.nanstd(abs_errors)
    
    mean_area = np.nanmean(row['proportional_changes_sampled'])
    sds = [np.sqrt(p*(1-p)) for p in row['proportional_changes_sampled']]
    mean_sd = np.nanmean(sds)
    se_area = np.divide(sd_area, np.sqrt(len(row['proportional_changes_sampled'])))
    
    #sd_area = np.nanstd(row['proportional_changes_sampled'])
    #se_area = np.divide(sd_area, np.sqrt(len(row['proportional_changes_sampled'])))
    #se_area = np.divide(sd_area, np.sqrt(row['sample_size']))
    
    ci = 1.96 * se_area
    perc_uncertainty = ci/mean_area*100
    
    return mean_deviation, sd_deviation, mean_area, mean_sd, se_area, perc_uncertainty

In [ ]:
target = 0.1  ## relative
sample_size = calculate_sample_size_cochran(1129313, 372, target_precision=target)
print(sample_size)
calculate_error_cochran(1129313, 372, sample_size=sample_size)


In [ ]:
d = {}
for idx, sample_size in enumerate(range(10000, 500000, 10000)):
    change_error = calculate_error_cochran(total_area = area_dict['total_area'],
        subarea = area_dict['change_area'],
        z_score=1.0,
        sample_size=sample_size)
    
    forest_error = calculate_error_cochran(total_area = area_dict['total_area'],
        subarea = area_dict['forest_area'],
        z_score=1.645,
        sample_size=sample_size)
    
    grid_size = np.sqrt(area_dict['total_area']/sample_size)
    d[idx] = sample_size, forest_error, change_error, grid_size
    
    
df = pd.DataFrame.from_dict(d, orient='index')

#display(df)
df.columns = ['Sample Size', 'Theoretical Sampling Error (Forest)', 'Theoretical Sampling Error (Deforestation)', 'Grid Size']
max_error = 5 # in percentage

selected = df[df['Theoretical Sampling Error (Deforestation)'] < max_error].head(1)

In [ ]:
df

In [ ]:
selected_grid_size = int(np.floor(selected['Grid Size'])*1000)
selected_grid_size

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,7))
df.plot.scatter(y='Theoretical Sampling Error (Forest)', x='Sample Size', ax=axes[0], color='green')
df.plot.scatter(y='Theoretical Sampling Error (Deforestation)', x='Sample Size', ax=axes[0], color='orange')
selected.plot.scatter(y='Theoretical Sampling Error (Deforestation)', x='Sample Size', ax=axes[0], color='blue')
axes[0].legend(['Forest stable', 'Forest Change', 'Selected'])
df.plot(kind='scatter', x='Sample Size', y='Grid Size', ax=axes[1],color='white')
selected.plot(kind='scatter', x='Sample Size', y='Grid Size', ax=axes[1], color='blue')
axes[0].set_facecolor("gainsboro")
axes[0].grid(color='white')
axes[1].set_facecolor("gainsboro")
axes[1].grid()

In [4]:
@retry(stop_max_attempt_number=5, wait_random_min=5000, wait_random_max=15000)
def get_sampling_errors(aoi, start, end, collection, scale, proportional_change, nr_of_runs_per_grid, grid_sizes, random_seed):    
    print(proportional_change)
    # create random seeds
    np.random.seed(random_seed)
    seeds = np.random.random(nr_of_runs_per_grid)
    seeds = list(np.round(np.multiply(seeds, 100), 0))
    
    if collection == 'CCI':

        lc = ee.ImageCollection("users/amitghosh/sdg_module/esa/cci_landcover")
        lc_start = lc.filter(ee.Filter.eq("system:index", f'{start}')).first()
        lc_end = lc.filter(ee.Filter.eq("system:index", f'{end}')).first()
        change = lc_start.neq(lc_end).clip(aoi)
        scale = 300

    elif collection == 'GFC':

        hansen = ee.Image("UMD/hansen/global_forest_change_2021_v1_9").select('lossyear').unmask(0)
        hansen = hansen.updateMask(hansen.mask().eq(1))
        change = hansen.gte(ee.Number(start).subtract(2000)).And(hansen.lte(ee.Number(end).subtract(2000)))
        scale = scale

    # -----------------------------------------------------------------
    # nested function for getting proportional change per grid size
    def get_grid_sample_error(grid):
        
        # set pixel size
        proj = ee.Projection("EPSG:3857").atScale(grid)
        
        # get total sample size
        sample_size = ee.Image(1).rename('sample_size').reproject(proj).reduceRegion(**{
            'reducer': ee.Reducer.sum(),
            'geometry': aoi,
            'scale': grid,
            'maxPixels': 1e14
        }).get('sample_size')
        
        
        # -----------------------------------------------------------------
        # nested function for getting proportional change per seed and grid
        def get_sampled_proportional_change(seed, proj):

            # create a subsample of our change image
            cells = ee.Image.random(seed).multiply(1000000).int().reproject(proj)
            random = ee.Image.random(seed).multiply(1000000).int()
            maximum = cells.addBands(random).reduceConnectedComponents(ee.Reducer.max())
            points = random.eq(maximum).selfMask().clip(aoi).reproject(proj.atScale(scale))

            # create a stack with change and total pixels as 1
            stack = (change.updateMask(points)          # masked sample change
                .addBands(points)                       # all samples
                .multiply(ee.Image.pixelArea())         # multiply both for pixel area
                .rename(['sampled_change', 'sampled_area'])
            )

            # sum them up
            areas = stack.reduceRegion(**{
                'reducer': ee.Reducer.sum(),
                'geometry': aoi,
                'scale': scale,
                'maxPixels': 1e14
            })

            # calculate proportional change to entire sampled area
            proportional_change_sampled = ee.Number(areas.get('sampled_change')).divide(ee.Number(areas.get('sampled_area'))).getInfo()
                        
            return proportional_change_sampled
        # -----------------------------------------------------------------
        
        # get sample error mean and stddev
        proportional_changes_sampled = [get_sampled_proportional_change(seed, proj) for seed in seeds]
        
        # add to a dict of all grids
        return proportional_changes_sampled, sample_size.getInfo()
    
    d, dfs = {}, []
    # we map over all different grid sizes
    print(f" Running the sampling error simulation. Please be patient, this can take a while.")
    for idx, grid in enumerate(grid_sizes):
        print(f" Running {nr_of_runs_per_grid} times the sample error simulation at a scale of {grid} meter.")
        proportional_changes_sampled, sample_size = get_grid_sample_error(grid)
        d['idx'] = idx
        d['spacing'] = grid
        d['sample_size'] = sample_size
        d['proportional_changes_sampled'] = proportional_changes_sampled
        dfs.append(pd.DataFrame([d]))
    
    return pd.concat(dfs)

SUM over no runs (cproport change of individual run- mean proport change over all runs)**2/(no runs -1)

In [ ]:
from pathlib import Path

def add_stats(row, actual_proportional):
    
    abs_errors = [np.abs(np.subtract(i, actual_proportional)) for i in row['proportional_changes_sampled']]
    mean_deviation, sd_deviation = np.nanmean(abs_errors), np.nanstd(abs_errors)
    
    mean_area = np.nanmean(row['proportional_changes_sampled']), 
    sd_area = np.nanstd(row['proportional_changes_sampled'])
    se_area = np.divide(sd_area, np.sqrt(len(row['proportional_changes_sampled'])))
    proportional_se = np.sqrt(sd_area**2)/actual_proportional*100
    
    return mean_deviation, sd_deviation, mean_area, sd_area, se_area, proportional_se

# based on earth engine feature collection
country = 'Zambia'


for country in [
    #'Liberia', 'Ethiopia', 'Kenya', 'Zambia', #'Uganda', 
    #'Ecuador', 
    #'Paraguay', 'Guyana', #'Peru', 'Colombia'
    #'Fiji', 'Viet Nam', 'Ghana', 
    'Cambodia', 'Bangladesh', 'Papua New Guinea'
]:
    
    print(country)
    # get AOI
    gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")
    aoi = gaul.filter(ee.Filter.eq("ADM0_NAME", country)).union()
    
    # get actual Hansen areas
    area_dict = get_aoi_areas(aoi, 'GFC', start_year, end_year, tree_cover)
    
    # do sampling simulation at different scales
    for scale in [30, 70, 100, 250]:
        
        if Path(f'{country}_25_runs_{scale}_scale.pickle').exists():
            continue
            
        grid_sizes = [500, 1000, 2000, 4000, 7500, 10000]
        df = get_sampling_errors(
            aoi, start_year, end_year, 'GFC', scale, 
            area_dict['change_area']/area_dict['total_area'], 
            25, grid_sizes, 7
        )
        df.to_pickle(f'{country}_25_runs_{scale}_scale.pickle')


Cambodia
According to the GFC product, the Area of Interest covers an area of 181423.76 square kilometers, of which 78371.69 square kilometers have been forested in 2015 (43.2 %). Between 2015 and 2020, 10833.91 square kilometers have been deforested. That corresponds to 1805.6516666666666 square kilometers of annual deforestation in average.
0.05971604821771966
 Running the sampling error simulation. Please be patient, this can take a while.
 Running 25 times the sample error simulation at a scale of 500 meter.
 Running 25 times the sample error simulation at a scale of 1000 meter.
 Running 25 times the sample error simulation at a scale of 2000 meter.
 Running 25 times the sample error simulation at a scale of 4000 meter.
 Running 25 times the sample error simulation at a scale of 7500 meter.
 Running 25 times the sample error simulation at a scale of 10000 meter.
0.05971604821771966
 Running the sampling error simulation. Please be patient, this can take a while.
 Running 25 times t

In [ ]:


actual_proportion = area_dict['change_area']/area_dict['total_area']
print(actual_proportion)
df = pd.read_pickle('zambia_10runs.pickle')
df[['mean', 'sd', 'mean_area', 'sd_area', 'se_area', 'proportional_error']] = df.apply(lambda x: add_stats(x, actual_proportion), axis=1, result_type='expand')

In [ ]:
display(df)

#fig, ax = plt.subplots()
ax = df.plot(x='sample_size', y='proportional_error',  kind='scatter')
#ax.set_xscale('log')
ax.ticklabel_format(useOffset=False, style='plain')